In [1]:
from random import seed
from random import random
import numpy as np
 
# 네트워크 초기 설정
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network
 
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [2]:
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i] # 순전파 진행 
        # 각 가중치와 input을 곱한 결과를 더합니다.
    return activation

def sigmoid(activation):
    return 1 / (1 + np.exp(-activation)) # 시그모이드 구현

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)  
            neuron['output'] = sigmoid(activation) # 나온 계산 값을 그대로 쓰나요?
            # sigmoid(활성함수)를 적용해서 신경망의 출력공간을 비선형으로 만들고 0과 1사이의 값으로 변환해줍니다.
            new_inputs.append(neuron['output']) # new_input은 다음 히든층에 들어갈 값이죠?
            # 전 layer에서 나온 output을 다시 다음 layer의 input으로!
        inputs = new_inputs
    return inputs

**여기까지는 순전파 학습과정이었습니다. 이 과정이 끝나면 가중치가 바뀌나요?  
답변을 답변의 근거 코딩 결과와 함께 보여주세요.**

In [3]:
for layer in network:
    for neuron in layer:
        print(neuron['weights'])
        
"""
** 원래 각 layer의 가중치 **
[0.13436424411240122, 0.8474337369372327, 0.763774618976614]
[0.2550690257394217, 0.49543508709194095]
[0.4494910647887381, 0.651592972722763]

** 확인한 layer의 가중치 **
[0.13436424411240122, 0.8474337369372327, 0.763774618976614]
[0.2550690257394217, 0.49543508709194095]
[0.4494910647887381, 0.651592972722763]
"""

"""
순전파 학습으로 인해 가중치는 바뀌지 않았다는 것을 알 수 있습니다.
순전파는 input에서 전달되는 값이 hidden layer를 거쳐 output까지 전달되는 방식인데,
순서대로 다음 층으로 이동하므로 결과를 이용해서 가중치를 update할 수 없습니다.

반대로 역전파는 최적의 가중치를 찾기 위해서 손실함수 값을 작게하는 가중치 값을 찾게 되므로
역전파 과정에서 가중치가 바뀌게 되고,
바뀐 가중치 값을 이용해서 순전파 과정에서 결과값을 출력하게 됩니다.
"""

[0.13436424411240122, 0.8474337369372327, 0.763774618976614]
[0.2550690257394217, 0.49543508709194095]
[0.4494910647887381, 0.651592972722763]


In [4]:
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


In [5]:
def sigmoid_derivative(output):
    return output * (1 - output) # 시그모이드 미분
    # d/dx(sigmoid(x)) = sigmoid(x) * (1 - sigmoid(x))

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error) 
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output']) # 역전파시 오차는 어떻게 설정했나요?
                # 예측값과 실제값의 차이를 오차로 설정하였습니다.
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * sigmoid_derivative(neuron['output']) # 시그모이드 함수를 사용한 역전파
            # 오차와 활성함수 미분값을 곱한 값 입니다.

In [6]:
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.002711797799238243}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887, 'delta': -0.14813473120687762}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748, 'delta': 0.05472601157879688}]


In [7]:
def weights_update(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta'] # 퍼셉트론 학습 규칙
            # 델타 규칙: learning rate * gradient
            

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row) # 순전파 
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i] - outputs[i])**2 for i in range(n_outputs)]) # 예측값의 오차 합
            # 평균제곱오차를 계산합니다
            backward_propagate_error(network, expected)
            weights_update(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [8]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

In [9]:
n_inputs = len(dataset[0]) - 1 # 뉴럴렛의 입력노드로 뭐가 들어가죠? 그럼 입력 노드의 개수는?
# dataset의 각 row가 input이 됩니다.
# 입력노드 개수는 특징의 개수이므로 위처럼 나타낼 수 있습니다.
n_outputs = len(set([row[-1] for row in dataset])) # 뉴럴렛의 출력노드의 개수는 뭐라고 했죠? 
# 부류 개수! 즉, dataset내의 class값의 수가 됩니다
network = initialize_network(n_inputs, 2, n_outputs)

train_network(network, dataset, 0.5, 20, n_outputs) # 자유롭게 설정하고 최적을 찾아보세요.

# 학습된(최적화)된 네트워크가 초기 네트워크와 달리 어떻게 변하였는지 출력하시오. (layer별로, hint : for문))
for layer in network:
    print(layer)

>epoch=0, lrate=0.500, error=6.350
>epoch=1, lrate=0.500, error=5.531
>epoch=2, lrate=0.500, error=5.221
>epoch=3, lrate=0.500, error=4.951
>epoch=4, lrate=0.500, error=4.519
>epoch=5, lrate=0.500, error=4.173
>epoch=6, lrate=0.500, error=3.835
>epoch=7, lrate=0.500, error=3.506
>epoch=8, lrate=0.500, error=3.192
>epoch=9, lrate=0.500, error=2.898
>epoch=10, lrate=0.500, error=2.626
>epoch=11, lrate=0.500, error=2.377
>epoch=12, lrate=0.500, error=2.153
>epoch=13, lrate=0.500, error=1.953
>epoch=14, lrate=0.500, error=1.774
>epoch=15, lrate=0.500, error=1.614
>epoch=16, lrate=0.500, error=1.472
>epoch=17, lrate=0.500, error=1.346
>epoch=18, lrate=0.500, error=1.233
>epoch=19, lrate=0.500, error=1.132
[{'weights': [-1.4688375095432327, 1.850887325439514, 1.0858178629550297], 'output': 0.029980305604426185, 'delta': -0.0059546604162323625}, {'weights': [0.37711098142462157, -0.0625909894552989, 0.2765123702642716], 'output': 0.9456229000211323, 'delta': 0.0026279652850863837}]
[{'weights

In [10]:
# 학습한 네트워크로 예측값을 뽑아보자.

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs)) # 순전파 결과에서 어떤것이 최종 아웃풋이 되나요?
    # 활성함수를 거쳐 나온 결과 값이 더 큰 class를 예측값으로 설정합니다

In [11]:
# 네트워크가 잘 학습되었는지 확인해보자. 

for row in dataset:
    prediction = predict(network, row) # 앞서 최적(학습)시킨 네트워크로 잘 학습되었는지 평가 
    # 앞에서 만든 predict 함수로 예측값을 뽑아봅니다
    print('실제값=%d, 예측값=%d' % (row[-1], prediction))
    # 각 row의 마지막 값이 실제값이므로
    
# 실제값과 예측값이 모두 동일하므로 잘 학습되었다는 것을 알 수 있습니다!

실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1


In [12]:
# 다른 learning rate, epoch으로 설정해봅시다

n_inputs = len(dataset[0]) - 1 # 뉴럴렛의 입력노드로 뭐가 들어가죠? 그럼 입력 노드의 개수는?
# dataset의 각 row가 input이 됩니다.
# 입력노드 개수는 특징의 개수이므로 위처럼 나타낼 수 있습니다.
n_outputs = len(set([row[-1] for row in dataset])) # 뉴럴렛의 출력노드의 개수는 뭐라고 했죠? 
# 부류 개수! 즉, dataset내의 class값의 수가 됩니다
network = initialize_network(n_inputs, 2, n_outputs)

train_network(network, dataset, 0.2, 20, n_outputs) # 자유롭게 설정하고 최적을 찾아보세요.

for row in dataset:
    prediction = predict(network, row) # 앞서 최적(학습)시킨 네트워크로 잘 학습되었는지 평가 
    # 앞에서 만든 predict 함수로 예측값을 뽑아봅니다
    print('실제값=%d, 예측값=%d' % (row[-1], prediction))
    # 각 row의 마지막 값이 실제값이므로
    
# 충분하지 않은 학습으로 인해 실제값과 예측값이 다른 경우가 많이 생겼습니다.
# learning rate가 작아지면 학습의 속도가 더디므로 적당한 learning rate의 설정이 중요하다는 것을 알 수 있습니다.

>epoch=0, lrate=0.200, error=6.657
>epoch=1, lrate=0.200, error=6.257
>epoch=2, lrate=0.200, error=5.921
>epoch=3, lrate=0.200, error=5.662
>epoch=4, lrate=0.200, error=5.480
>epoch=5, lrate=0.200, error=5.360
>epoch=6, lrate=0.200, error=5.286
>epoch=7, lrate=0.200, error=5.240
>epoch=8, lrate=0.200, error=5.211
>epoch=9, lrate=0.200, error=5.192
>epoch=10, lrate=0.200, error=5.178
>epoch=11, lrate=0.200, error=5.167
>epoch=12, lrate=0.200, error=5.157
>epoch=13, lrate=0.200, error=5.148
>epoch=14, lrate=0.200, error=5.139
>epoch=15, lrate=0.200, error=5.130
>epoch=16, lrate=0.200, error=5.119
>epoch=17, lrate=0.200, error=5.108
>epoch=18, lrate=0.200, error=5.094
>epoch=19, lrate=0.200, error=5.078
실제값=0, 예측값=1
실제값=0, 예측값=1
실제값=0, 예측값=1
실제값=0, 예측값=1
실제값=0, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1


In [17]:
# 다른 learning rate, epoch으로 설정해봅시다

n_inputs = len(dataset[0]) - 1 # 뉴럴렛의 입력노드로 뭐가 들어가죠? 그럼 입력 노드의 개수는?
# dataset의 각 row가 input이 됩니다.
# 입력노드 개수는 특징의 개수이므로 위처럼 나타낼 수 있습니다.
n_outputs = len(set([row[-1] for row in dataset])) # 뉴럴렛의 출력노드의 개수는 뭐라고 했죠? 
# 부류 개수! 즉, dataset내의 class값의 수가 됩니다
network = initialize_network(n_inputs, 2, n_outputs)

train_network(network, dataset, 0.5, 40, n_outputs) # 자유롭게 설정하고 최적을 찾아보세요.

for row in dataset:
    prediction = predict(network, row) # 앞서 최적(학습)시킨 네트워크로 잘 학습되었는지 평가 
    # 앞에서 만든 predict 함수로 예측값을 뽑아봅니다
    print('실제값=%d, 예측값=%d' % (row[-1], prediction))
    # 각 row의 마지막 값이 실제값이므로
    
# 더 많은 epoch을 진행하면서 error가 더 작아졌음을 알 수 있습니다.
# 실제값과 예측값 또한 모두 동일합니다.

>epoch=0, lrate=0.500, error=7.326
>epoch=1, lrate=0.500, error=6.387
>epoch=2, lrate=0.500, error=5.630
>epoch=3, lrate=0.500, error=5.316
>epoch=4, lrate=0.500, error=5.256
>epoch=5, lrate=0.500, error=5.235
>epoch=6, lrate=0.500, error=5.181
>epoch=7, lrate=0.500, error=5.019
>epoch=8, lrate=0.500, error=4.689
>epoch=9, lrate=0.500, error=4.317
>epoch=10, lrate=0.500, error=3.928
>epoch=11, lrate=0.500, error=3.510
>epoch=12, lrate=0.500, error=3.088
>epoch=13, lrate=0.500, error=2.686
>epoch=14, lrate=0.500, error=2.322
>epoch=15, lrate=0.500, error=2.002
>epoch=16, lrate=0.500, error=1.729
>epoch=17, lrate=0.500, error=1.499
>epoch=18, lrate=0.500, error=1.306
>epoch=19, lrate=0.500, error=1.146
>epoch=20, lrate=0.500, error=1.012
>epoch=21, lrate=0.500, error=0.900
>epoch=22, lrate=0.500, error=0.806
>epoch=23, lrate=0.500, error=0.726
>epoch=24, lrate=0.500, error=0.658
>epoch=25, lrate=0.500, error=0.601
>epoch=26, lrate=0.500, error=0.551
>epoch=27, lrate=0.500, error=0.508
>e